In [1]:
import pandas as pd
import numpy as np
import os
import random

from mlchartist.array_builder import build_arrays, build_randomised_arrays
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [4]:
df = pd.read_csv('../../raw_data/processed/aapl.csv')

In [5]:
df['date'] = pd.to_datetime(df['date'])

In [50]:
FIVE_TR = 0.0006
TEN_TR = 0.0012
TWENTY_TR = 0.0024
INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']
#INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal']

In [51]:
df['5D_return_bin'] = (df['5TD_return'] >= FIVE_TR)
df['10D_return_bin'] = (df['10TD_return'] >= TEN_TR)
df['20D_return_bin'] = (df['20TD_return'] >= TWENTY_TR)

In [52]:
test_df = df[(df['date'].dt.year >= 2017) & (df['date'].dt.year <= 2019)]
train_df = df[(df['date'].dt.year >= 2000) & (df['date'].dt.year < 2017)]

In [53]:
scaler = StandardScaler()

In [54]:
scaler.fit(train_df[INPUT_COLS])

StandardScaler()

In [55]:
train_df.loc[:, INPUT_COLS] = scaler.transform(train_df[INPUT_COLS])
test_df.loc[:, INPUT_COLS] = scaler.transform(test_df[INPUT_COLS])

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [56]:
from sklearn.decomposition import PCA

In [57]:
pca = PCA(n_components=9)
pca.fit(train_df[INPUT_COLS])

PCA(n_components=9)

In [58]:
train_pca = pca.transform(train_df[INPUT_COLS])

In [59]:
test_pca = pca.transform(test_df[INPUT_COLS])

In [60]:
pca.explained_variance_ratio_

array([0.43047229, 0.18917887, 0.11682235, 0.06885011, 0.06577291,
       0.03995869, 0.0218288 , 0.01895745, 0.01589968])

In [61]:
col_names = ['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9']

In [62]:
INPUT_COLS_PCA = ['PCA1', 'PCA2', 'PCA3']

In [63]:
train_df.loc[:, col_names] = train_pca
test_df.loc[:, col_names] = test_pca

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [64]:
apple_train_x_20, apple_train_y_20 = build_arrays(train_df,input_cols=INPUT_COLS_PCA, target_col='20D_return_bin', time_window=40, stride=1)
apple_test_x_20, apple_test_y_20 = build_arrays(test_df,input_cols=INPUT_COLS_PCA, target_col='20D_return_bin', time_window=40, stride=1)

In [65]:
indx = list(range(len(apple_train_x_20)))

In [66]:
len(indx)

4236

In [67]:
test_df.shape

(754, 32)

In [68]:
sample_indx = random.sample(indx, 4200)

In [69]:
X_train_20 =  apple_train_x_20[[sample_indx], :][0]

In [70]:
y_train_20 = apple_train_y_20[[sample_indx]]

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [71]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers, models 
from tensorflow.keras.optimizers import RMSprop, Adam

optim = RMSprop(learning_rate=0.0001)

def init_model():
    model = Sequential()
    model.add(layers.LSTM(100, return_sequences=True, input_shape=(40, 3), activation='tanh'))
    model.add(layers.LSTM(100, activation='tanh'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    
    return model

### Model for 20 days future returns

In [72]:
model_20 = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model_20.fit(X_train_20, y_train_20, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
          callbacks=[es])

Epoch 1/500
210/210 [==============================] - 2s 9ms/step - loss: 0.6932 - accuracy: 0.5301 - val_loss: 0.6742 - val_accuracy: 0.6083
Epoch 2/500
210/210 [==============================] - 1s 4ms/step - loss: 0.6834 - accuracy: 0.5735 - val_loss: 0.6699 - val_accuracy: 0.6226
Epoch 3/500
210/210 [==============================] - 1s 7ms/step - loss: 0.6779 - accuracy: 0.5738 - val_loss: 0.6624 - val_accuracy: 0.6036
Epoch 4/500
210/210 [==============================] - 1s 7ms/step - loss: 0.6760 - accuracy: 0.5839 - val_loss: 0.6573 - val_accuracy: 0.6083
Epoch 5/500
210/210 [==============================] - 1s 6ms/step - loss: 0.6772 - accuracy: 0.5732 - val_loss: 0.6569 - val_accuracy: 0.6226
Epoch 6/500
210/210 [==============================] - 1s 6ms/step - loss: 0.6727 - accuracy: 0.5911 - val_loss: 0.6534 - val_accuracy: 0.6250
Epoch 7/500
210/210 [==============================] - 1s 7ms/step - loss: 0.6733 - accuracy: 0.5878 - val_loss: 0.6567 - val_accuracy: 0.6167

In [73]:
apple_train_y_20.sum()/len(apple_train_y_20)

0.5880547686496695

In [74]:
apple_test_y_20.sum()/len(apple_test_y_20)

0.6853146853146853

In [76]:
model_20.evaluate(apple_test_x_20, apple_test_y_20)

23/23 [==============================] - 0s 3ms/step - loss: 2.6741 - accuracy: 0.3930


[2.67409086227417, 0.39300698041915894]

### Model for 10 days future returns

In [80]:
apple_train_x_10, apple_train_y_10 = build_arrays(train_df,input_cols=INPUT_COLS_PCA, target_col='10D_return_bin', time_window=40, stride=1)
apple_test_x_10, apple_test_y_10 = build_arrays(test_df,input_cols=INPUT_COLS_PCA, target_col='10D_return_bin', time_window=40, stride=1)

In [81]:
X_train_10 =  apple_train_x_10[[sample_indx], :][0]
y_train_10 = apple_train_y_10[[sample_indx]]

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [82]:
model_10 = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model_10.fit(X_train_10, y_train_10, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
          callbacks=[es])

Epoch 1/500
210/210 [==============================] - 2s 7ms/step - loss: 0.6910 - accuracy: 0.5381 - val_loss: 0.6818 - val_accuracy: 0.5786
Epoch 2/500
210/210 [==============================] - 1s 5ms/step - loss: 0.6882 - accuracy: 0.5414 - val_loss: 0.6809 - val_accuracy: 0.5821
Epoch 3/500
210/210 [==============================] - 1s 6ms/step - loss: 0.6884 - accuracy: 0.5491 - val_loss: 0.6821 - val_accuracy: 0.5762
Epoch 4/500
210/210 [==============================] - 1s 6ms/step - loss: 0.6876 - accuracy: 0.5509 - val_loss: 0.6801 - val_accuracy: 0.5774
Epoch 5/500
210/210 [==============================] - 1s 6ms/step - loss: 0.6837 - accuracy: 0.5673 - val_loss: 0.6807 - val_accuracy: 0.5869
Epoch 6/500
210/210 [==============================] - 1s 5ms/step - loss: 0.6849 - accuracy: 0.5568 - val_loss: 0.6813 - val_accuracy: 0.5857
Epoch 7/500
210/210 [==============================] - 1s 5ms/step - loss: 0.6835 - accuracy: 0.5574 - val_loss: 0.6797 - val_accuracy: 0.5869

In [83]:
apple_test_y_10.sum()/len(apple_test_y_10)

0.641958041958042

In [84]:
model_10.evaluate(apple_test_x_10, apple_test_y_10)

23/23 [==============================] - 0s 2ms/step - loss: 1.1341 - accuracy: 0.5259


[1.1340992450714111, 0.525874137878418]

### Model for 5 days future returns

In [85]:
apple_train_x_5, apple_train_y_5 = build_arrays(train_df,input_cols=INPUT_COLS_PCA, target_col='5D_return_bin', time_window=40, stride=1)
apple_test_x_5, apple_test_y_5 = build_arrays(test_df,input_cols=INPUT_COLS_PCA, target_col='5D_return_bin', time_window=40, stride=1)

In [86]:
X_train_5 =  apple_train_x_5[[sample_indx], :][0]
y_train_5 = apple_train_y_5[[sample_indx]]

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [87]:
model_5 = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model_5.fit(X_train_5, y_train_5, 
          epochs=500, 
          batch_size=8,
          validation_split=0.2,
          callbacks=[es])

Epoch 1/500
420/420 [==============================] - 2s 6ms/step - loss: 0.6922 - accuracy: 0.5333 - val_loss: 0.6878 - val_accuracy: 0.5488
Epoch 2/500
420/420 [==============================] - 3s 6ms/step - loss: 0.6922 - accuracy: 0.5360 - val_loss: 0.6856 - val_accuracy: 0.5524
Epoch 3/500
420/420 [==============================] - 3s 7ms/step - loss: 0.6903 - accuracy: 0.5452 - val_loss: 0.6863 - val_accuracy: 0.5560
Epoch 4/500
420/420 [==============================] - 3s 7ms/step - loss: 0.6880 - accuracy: 0.5387 - val_loss: 0.6852 - val_accuracy: 0.5583
Epoch 5/500
420/420 [==============================] - 3s 8ms/step - loss: 0.6902 - accuracy: 0.5354 - val_loss: 0.6862 - val_accuracy: 0.5607
Epoch 6/500
420/420 [==============================] - 3s 7ms/step - loss: 0.6898 - accuracy: 0.5452 - val_loss: 0.6870 - val_accuracy: 0.5536
Epoch 7/500
420/420 [==============================] - 3s 7ms/step - loss: 0.6860 - accuracy: 0.5613 - val_loss: 0.6879 - val_accuracy: 0.5381

In [45]:
apple_test_y_5.sum()/len(apple_test_y_5)

0.6111888111888112

In [88]:
model_5.evaluate(apple_test_x_5, apple_test_y_5)

23/23 [==============================] - 0s 3ms/step - loss: 0.6766 - accuracy: 0.5846


[0.6765639185905457, 0.5846154093742371]

# CNN test

### CNN for 20 days future returns

In [288]:
from tensorflow.keras.backend import expand_dims
X_train_cnn_20 = expand_dims(X_train_20, axis=-1)
X_test_cnn_20 = expand_dims(apple_test_x_20, axis=-1)


In [289]:
def initialize_model_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(40, 15, 1)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [290]:
model_cnn_20 = initialize_model_cnn()

model_cnn_20.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_20.fit(X_train_cnn_20, y_train_20, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
            callbacks=[es])

Epoch 1/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6852 - accuracy: 0.5670 - val_loss: 0.6656 - val_accuracy: 0.5830
Epoch 2/500
250/250 [==============================] - 0s 2ms/step - loss: 0.6724 - accuracy: 0.5767 - val_loss: 0.6574 - val_accuracy: 0.5900
Epoch 3/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6487 - accuracy: 0.6097 - val_loss: 0.6172 - val_accuracy: 0.6400
Epoch 4/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6152 - accuracy: 0.6495 - val_loss: 0.5857 - val_accuracy: 0.6950
Epoch 5/500
250/250 [==============================] - 1s 3ms/step - loss: 0.5692 - accuracy: 0.6965 - val_loss: 0.5227 - val_accuracy: 0.7250
Epoch 6/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4995 - accuracy: 0.7655 - val_loss: 0.5045 - val_accuracy: 0.7340
Epoch 7/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4520 - accuracy: 0.7950 - val_loss: 0.4304 - val_accuracy: 0.8150

In [291]:
model_cnn_20.evaluate(X_test_cnn_20, apple_test_y_20)

23/23 [==============================] - 0s 2ms/step - loss: 5.2828 - accuracy: 0.4364 


[5.282845497131348, 0.4363636374473572]

### CNN for 10 days future returns

In [292]:
X_train_cnn_10 = expand_dims(X_train_10, axis=-1)
X_test_cnn_10 = expand_dims(apple_test_x_10, axis=-1)

In [294]:
model_cnn_10 = initialize_model_cnn()

model_cnn_10.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_10.fit(X_train_cnn_10, y_train_10, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
            callbacks=[es])

Epoch 1/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6943 - accuracy: 0.5315 - val_loss: 0.6884 - val_accuracy: 0.5670
Epoch 2/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6887 - accuracy: 0.5540 - val_loss: 0.6843 - val_accuracy: 0.5640
Epoch 3/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6840 - accuracy: 0.5497 - val_loss: 0.6807 - val_accuracy: 0.5780
Epoch 4/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6793 - accuracy: 0.5675 - val_loss: 0.6742 - val_accuracy: 0.5850
Epoch 5/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6687 - accuracy: 0.5817 - val_loss: 0.6578 - val_accuracy: 0.6150
Epoch 6/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6559 - accuracy: 0.6202 - val_loss: 0.6360 - val_accuracy: 0.6080
Epoch 7/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6284 - accuracy: 0.6388 - val_loss: 0.6216 - val_accuracy: 0.6560

In [295]:
model_cnn_10.evaluate(X_test_cnn_10, apple_test_y_10)

23/23 [==============================] - 0s 1ms/step - loss: 2.2595 - accuracy: 0.5832


[2.2594521045684814, 0.5832167863845825]

### CNN for 5 days future returns

In [296]:
X_train_cnn_5 = expand_dims(X_train_5, axis=-1)
X_test_cnn_5 = expand_dims(apple_test_x_5, axis=-1)

In [297]:
model_cnn_5 = initialize_model_cnn()

model_cnn_5.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_5.fit(X_train_cnn_5, y_train_5, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
            callbacks=[es])

Epoch 1/500
250/250 [==============================] - 1s 4ms/step - loss: 0.6953 - accuracy: 0.5163 - val_loss: 0.6879 - val_accuracy: 0.5620
Epoch 2/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6920 - accuracy: 0.5335 - val_loss: 0.6886 - val_accuracy: 0.5620
Epoch 3/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6915 - accuracy: 0.5415 - val_loss: 0.6869 - val_accuracy: 0.5620
Epoch 4/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6911 - accuracy: 0.5390 - val_loss: 0.6857 - val_accuracy: 0.5620
Epoch 5/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6894 - accuracy: 0.5347 - val_loss: 0.6818 - val_accuracy: 0.5660
Epoch 6/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6887 - accuracy: 0.5378 - val_loss: 0.6818 - val_accuracy: 0.5650
Epoch 7/500
250/250 [==============================] - 0s 2ms/step - loss: 0.6838 - accuracy: 0.5428 - val_loss: 0.6803 - val_accuracy: 0.5630

In [298]:
model_cnn_5.evaluate(X_test_cnn_5, apple_test_y_5)

23/23 [==============================] - 0s 1ms/step - loss: 0.9555 - accuracy: 0.4965


[0.9554838538169861, 0.4965035021305084]